---

# 🔧 Appendice: Algoritmo di Shor Generalizzato

Questa sezione implementa l'algoritmo di Shor in modo **completamente parametrico**, permettendo di scegliere qualsiasi coppia $(a, N)$ con $\gcd(a, N) = 1$.

## Il Problema dell'Implementazione Standard

Nel notebook principale, la funzione `c_amod15` è **hardcoded** per N=15. Per generalizzare, dobbiamo implementare l'operazione:

$$|x\rangle \mapsto |a \cdot x \mod N\rangle$$

in modo generale.

## Approcci Possibili

1. **Matrice di Permutazione**: Costruisce esplicitamente l'operatore unitario. Semplice ma limitato a N piccoli (~64-128).

2. **Circuiti Aritmetici**: Usa adder modulari (Draper, VBE). Scala meglio ma circuiti molto profondi.

3. **Approccio Ibrido**: Combina simulazione classica per l'oracolo con QPE quantistica.

## Metodo 1: Matrice di Permutazione

L'operazione $|x\rangle \mapsto |a \cdot x \mod N\rangle$ è una **permutazione** degli stati base. Possiamo costruire esplicitamente la matrice unitaria.

In [1]:
# Imports necessari
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.circuit.library import UnitaryGate
from qiskit.visualization import plot_histogram
import numpy as np
from math import gcd, ceil, log2
from fractions import Fraction
from typing import Tuple, List, Optional

print("✓ Imports per Shor generalizzato")

✓ Imports per Shor generalizzato


In [2]:
def modular_multiplication_matrix(a: int, N: int) -> np.ndarray:
    """
    Costruisce la matrice di permutazione per la moltiplicazione modulare.
    
    L'operazione è: |x⟩ → |a·x mod N⟩ per x < N
                    |x⟩ → |x⟩        per x >= N (stati non usati)
    
    Args:
        a: Moltiplicatore (deve essere coprimo con N)
        N: Modulo
    
    Returns:
        Matrice unitaria di dimensione 2^n × 2^n dove n = ceil(log2(N))
    """
    if gcd(a, N) != 1:
        raise ValueError(f"a={a} deve essere coprimo con N={N}")
    
    # Numero di qubit necessari
    n_qubits = ceil(log2(N + 1))  # +1 per sicurezza
    dim = 2 ** n_qubits
    
    # Costruisci matrice di permutazione
    U = np.zeros((dim, dim), dtype=complex)
    
    for x in range(dim):
        if x < N:
            # Moltiplica mod N
            y = (a * x) % N
        else:
            # Stati fuori range: identità
            y = x
        U[y, x] = 1.0
    
    return U


def create_controlled_modmul_gate(a: int, power: int, N: int) -> UnitaryGate:
    """
    Crea il gate controllato per a^(2^power) mod N.
    
    Args:
        a: Base
        power: Esponente (il gate calcola a^(2^power) mod N)
        N: Modulo
    
    Returns:
        UnitaryGate controllato
    """
    # Calcola a^(2^power) mod N
    a_power = pow(a, 2**power, N)
    
    # Costruisci la matrice
    U_matrix = modular_multiplication_matrix(a_power, N)
    
    # Crea il gate
    gate = UnitaryGate(U_matrix, label=f'{a}^{2**power} mod {N}')
    
    return gate.control(1, label=f'c-{a}^{2**power}')


# Test
print("Test: Matrice per 7·x mod 15")
U = modular_multiplication_matrix(7, 15)
print(f"Dimensione matrice: {U.shape}")
print(f"È unitaria: {np.allclose(U @ U.conj().T, np.eye(16))}")

# Verifica alcune moltiplicazioni
print("\nVerifica:")
for x in range(5):
    # Trova dove va |x⟩
    y = np.argmax(U[:, x])
    expected = (7 * x) % 15
    print(f"  |{x}⟩ → |{y}⟩  (atteso: {expected}) {'✓' if y == expected else '✗'}")

Test: Matrice per 7·x mod 15
Dimensione matrice: (16, 16)
È unitaria: True

Verifica:
  |0⟩ → |0⟩  (atteso: 0) ✓
  |1⟩ → |7⟩  (atteso: 7) ✓
  |2⟩ → |14⟩  (atteso: 14) ✓
  |3⟩ → |6⟩  (atteso: 6) ✓
  |4⟩ → |13⟩  (atteso: 13) ✓


## Implementazione QPE Generalizzata per Shor

In [14]:
def inverse_qft(n: int) -> QuantumCircuit:
    """QFT inversa per n qubit."""
    qc = QuantumCircuit(n, name='QFT†')
    
    for j in range(n):
        for k in range(j):
            qc.cp(-np.pi / (2 ** (j - k)), k, j)
        qc.h(j)
    
    for i in range(n // 2):
        qc.swap(i, n - i - 1)


    return qc


def build_shor_circuit_general(a: int, N: int, n_count: int = None) -> QuantumCircuit:
    """
    Costruisce il circuito di Shor generalizzato per qualsiasi (a, N) coprimi.
    
    Args:
        a: Base per l'esponenziazione modulare
        N: Numero da fattorizzare
        n_count: Qubit di conteggio (default: 2*ceil(log2(N)))
    
    Returns:
        Circuito quantistico per trovare il periodo di a^x mod N
    """
    # Validazione
    if gcd(a, N) != 1:
        raise ValueError(f"a={a} e N={N} devono essere coprimi")
    
    if a >= N:
        raise ValueError(f"a={a} deve essere minore di N={N}")
    
    # Calcola qubit necessari
    n_target = ceil(log2(N + 1))  # Qubit per il registro target
    if n_count is None:
        n_count = 2 * n_target  # Precisione standard
    
    print(f"Configurazione circuito Shor:")
    print(f"  N = {N}, a = {a}")
    print(f"  Qubit conteggio: {n_count}")
    print(f"  Qubit target: {n_target}")
    print(f"  Qubit totali: {n_count + n_target}")
    
    # Registri
    count_reg = QuantumRegister(n_count, 'count')
    target_reg = QuantumRegister(n_target, 'target')
    meas_reg = ClassicalRegister(n_count, 'meas')
    
    qc = QuantumCircuit(count_reg, target_reg, meas_reg)
    
    # ═══════════════════════════════════════════════════════════════
    # PASSO 1: Inizializzazione
    # - Registro conteggio in superposizione uniforme
    # - Registro target in |1⟩ (perché a^0 = 1)
    # ═══════════════════════════════════════════════════════════════
    qc.h(count_reg)
    qc.x(target_reg[0])  # |1⟩ = |00...01⟩
    
    qc.barrier(label='Init')
    
    # ═══════════════════════════════════════════════════════════════
    # PASSO 2: Esponenziazione modulare controllata
    # |j⟩|1⟩ → |j⟩|a^j mod N⟩
    # 
    # Usando la decomposizione: a^j = a^(j₀·2⁰) · a^(j₁·2¹) · ... 
    # ═══════════════════════════════════════════════════════════════
    for k in range(n_count):
        # Crea gate per a^(2^k) mod N controllato
        c_gate = create_controlled_modmul_gate(a, k, N)
        
        # Applica: qubit di controllo è count[k], target è tutto il registro target
        qc.append(c_gate, [count_reg[k]] + list(target_reg))
    
    qc.barrier(label='ModExp')
    
    # ═══════════════════════════════════════════════════════════════
    # PASSO 3: QFT Inversa sul registro di conteggio
    # ═══════════════════════════════════════════════════════════════
    qft_inv = inverse_qft(n_count)
    qc.compose(qft_inv, count_reg, inplace=True)
    
    qc.barrier(label='QFT†')
    
    # ═══════════════════════════════════════════════════════════════
    # PASSO 4: Misura
    # ═══════════════════════════════════════════════════════════════
    qc.measure(count_reg, meas_reg)
    
    return qc


# Test: Shor per a=7, N=15
print("="*60)
qc_test = build_shor_circuit_general(a=7, N=15, n_count=8)
print(f"\nCircuito costruito:")
print(f"  Profondità: {qc_test.depth()}")
print(f"  Gate totali: {qc_test.size()}")

Configurazione circuito Shor:
  N = 15, a = 7
  Qubit conteggio: 8
  Qubit target: 4
  Qubit totali: 12

Circuito costruito:
  Profondità: 26
  Gate totali: 65


## Funzioni di Analisi e Post-Processing

In [15]:
def extract_period_from_phase(phase: float, N: int) -> Optional[int]:
    """
    Estrae il periodo dalla fase misurata usando frazioni continue.
    
    Args:
        phase: Fase misurata (s/2^n)
        N: Modulo (upper bound per il periodo)
    
    Returns:
        Periodo candidato o None
    """
    if phase == 0:
        return None
    
    # Usa frazioni continue per trovare s/r
    frac = Fraction(phase).limit_denominator(N)
    
    return frac.denominator


def find_factors(a: int, r: int, N: int) -> Tuple[Optional[int], Optional[int]]:
    """
    Trova i fattori di N dato il periodo r.
    
    Se r è pari e a^(r/2) ≠ -1 (mod N), allora:
    - gcd(a^(r/2) + 1, N) è un fattore
    - gcd(a^(r/2) - 1, N) è un fattore
    
    Args:
        a: Base usata
        r: Periodo trovato
        N: Numero da fattorizzare
    
    Returns:
        (fattore1, fattore2) o (None, None) se fallisce
    """
    # Verifica che r sia effettivamente il periodo
    if pow(a, r, N) != 1:
        return None, None
    
    # r deve essere pari
    if r % 2 == 1:
        return None, None
    
    # Calcola a^(r/2) mod N
    x = pow(a, r // 2, N)
    
    # a^(r/2) ≠ -1 (mod N)
    if x == N - 1:
        return None, None
    
    # Calcola i fattori
    f1 = gcd(x + 1, N)
    f2 = gcd(x - 1, N)
    
    # Verifica che siano fattori non banali
    if f1 == 1 or f1 == N:
        f1 = None
    if f2 == 1 or f2 == N:
        f2 = None
    
    return f1, f2


def analyze_shor_results(counts: dict, n_count: int, a: int, N: int) -> dict:
    """
    Analizza i risultati dell'algoritmo di Shor.
    
    Args:
        counts: Conteggi delle misure
        n_count: Numero di qubit di conteggio
        a: Base usata
        N: Numero da fattorizzare
    
    Returns:
        Dizionario con periodi, fasi, e fattori trovati
    """
    total_shots = sum(counts.values())
    
    print("═" * 70)
    print(f"ANALISI RISULTATI SHOR per a={a}, N={N}")
    print("═" * 70)
    
    # Calcola periodo classico per verifica
    r_true = 1
    temp = a
    while temp != 1:
        temp = (temp * a) % N
        r_true += 1
    print(f"\nPeriodo vero (classico): r = {r_true}")
    
    # Analizza misure
    print(f"\n{'Misura':<12} {'Decimale':<10} {'Fase':<12} {'Periodo':<10} {'Conteggi':<10}")
    print("-" * 60)
    
    periods_found = {}
    sorted_counts = sorted(counts.items(), key=lambda x: -x[1])
    
    for bitstring, count in sorted_counts[:12]:
        decimal = int(bitstring, 2)
        phase = decimal / (2 ** n_count)
        period = extract_period_from_phase(phase, N)
        
        if period and period > 0:
            if period not in periods_found:
                periods_found[period] = 0
            periods_found[period] += count
        
        period_str = str(period) if period else "-"
        print(f"{bitstring:<12} {decimal:<10} {phase:<12.4f} {period_str:<10} {count:<10}")
    
    # Riepilogo periodi
    print(f"\n{'─'*60}")
    print("Periodi trovati (ordinati per frequenza):")
    for r, cnt in sorted(periods_found.items(), key=lambda x: -x[1])[:5]:
        is_true = "✓" if r == r_true else ""
        is_valid = "(valido)" if pow(a, r, N) == 1 else "(non valido)"
        print(f"  r = {r}: {cnt} conteggi {is_valid} {is_true}")
    
    # Tenta fattorizzazione
    print(f"\n{'─'*60}")
    print("Tentativi di fattorizzazione:")
    
    factors_found = None
    for r in sorted(periods_found.keys(), key=lambda x: -periods_found[x]):
        f1, f2 = find_factors(a, r, N)
        if f1 or f2:
            factors_found = (f1, f2)
            print(f"  r = {r}: Fattori trovati! {f1} × {f2 if f2 else N//f1} = {N}")
            break
        else:
            reason = ""
            if pow(a, r, N) != 1:
                reason = "(r non è periodo)"
            elif r % 2 == 1:
                reason = "(r dispari)"
            else:
                reason = "(a^(r/2) ≡ -1)"
            print(f"  r = {r}: Fallito {reason}")
    
    if not factors_found:
        print("  ⚠ Nessun fattore trovato. Prova con un'altra base a.")
    
    return {
        'periods': periods_found,
        'true_period': r_true,
        'factors': factors_found
    }

## Funzione Completa: Shor Generalizzato

In [26]:
def run_shor_general(N: int, a: int = None, n_count: int = None, 
                     shots: int = 4096, verbose: bool = True) -> dict:
    """
    Esegue l'algoritmo di Shor completo per fattorizzare N.
    
    Args:
        N: Numero da fattorizzare (deve essere composto, non primo)
        a: Base per esponenziazione (se None, sceglie automaticamente)
        n_count: Qubit di conteggio (se None, usa 2*ceil(log2(N)))
        shots: Numero di misure
        verbose: Se stampare dettagli
    
    Returns:
        Dizionario con risultati completi
    """
    # ═══════════════════════════════════════════════════════════════
    # PARTE CLASSICA 1: Validazione e setup
    # ═══════════════════════════════════════════════════════════════
    
    # Verifica che N non sia troppo grande
    if N > 128:
        print(f"⚠ ATTENZIONE: N={N} potrebbe essere troppo grande.")
        print(f"  Il metodo della matrice di permutazione richiede 2^{ceil(log2(N))} elementi.")
        print(f"  Per N grandi, usa il metodo con circuiti aritmetici.")
    
    # Verifica che N sia composto
    def is_prime(n):
        if n < 2: return False
        for i in range(2, int(n**0.5) + 1):
            if n % i == 0: return False
        return True
    
    if is_prime(N):
        print(f"N={N} è primo! Non può essere fattorizzato.")
        return None
    
    # Verifica se N è potenza di primo
    for p in range(2, int(log2(N)) + 2):
        root = round(N ** (1/p))
        if root ** p == N:
            print(f"N={N} = {root}^{p} (potenza di primo)")
            return {'factors': (root, N // root), 'method': 'power_detection'}
    
    # ═══════════════════════════════════════════════════════════════
    # PARTE CLASSICA 2: Scelta della base a
    # ═══════════════════════════════════════════════════════════════
    
    if a is None:
        # Scegli a random coprimo con N
        import random
        candidates = [x for x in range(2, N) if gcd(x, N) == 1]
        a = random.choice(candidates)
        if verbose:
            print(f"Base scelta automaticamente: a = {a}")
    
    # Verifica se abbiamo trovato un fattore per caso
    g = gcd(a, N)
    if g > 1:
        if verbose:
            print(f"Fortunato! gcd({a}, {N}) = {g} è un fattore!")
        return {'factors': (g, N // g), 'method': 'gcd_lucky'}
    
    # ═══════════════════════════════════════════════════════════════
    # PARTE QUANTISTICA: Period Finding
    # ═══════════════════════════════════════════════════════════════
    
    if verbose:
        print(f"\n{'═'*60}")
        print(f"ALGORITMO DI SHOR GENERALIZZATO")
        print(f"{'═'*60}")
        print(f"Obiettivo: Fattorizzare N = {N}")
        print(f"Base: a = {a}")
    
    # Costruisci circuito
    qc = build_shor_circuit_general(a, N, n_count)
    actual_n_count = qc.num_clbits
    qc.draw('mpl', fold=-1)
    # Esegui
    if verbose:
        print(f"\nEsecuzione simulazione...")
    
    simulator = AerSimulator()
    sampler = SamplerV2()
    pm = generate_preset_pass_manager(optimization_level=1, backend=simulator)
    transpiled = pm.run(qc)
    
    job = sampler.run([transpiled], shots=shots)
    result = job.result()
    counts = result[0].data.meas.get_counts()
    
    # ═══════════════════════════════════════════════════════════════
    # PARTE CLASSICA 3: Analisi risultati
    # ═══════════════════════════════════════════════════════════════
    
    analysis = analyze_shor_results(counts, actual_n_count, a, N)
    
    return {
        'N': N,
        'a': a,
        'counts': counts,
        'analysis': analysis,
        'circuit': qc
    }

## Esempi di Utilizzo

In [27]:
# ═══════════════════════════════════════════════════════════════
# ESEMPIO 1: Fattorizzare 15 con a=7 (caso classico)
# ═══════════════════════════════════════════════════════════════

print("\n" + "▓"*70)
print("ESEMPIO 1: N = 15, a = 7")
print("▓"*70)

result_15 = run_shor_general(N=15, a=7, n_count=8, shots=4096)


▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
ESEMPIO 1: N = 15, a = 7
▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓

════════════════════════════════════════════════════════════
ALGORITMO DI SHOR GENERALIZZATO
════════════════════════════════════════════════════════════
Obiettivo: Fattorizzare N = 15
Base: a = 7
Configurazione circuito Shor:
  N = 15, a = 7
  Qubit conteggio: 8
  Qubit target: 4
  Qubit totali: 12

Esecuzione simulazione...
══════════════════════════════════════════════════════════════════════
ANALISI RISULTATI SHOR per a=7, N=15
══════════════════════════════════════════════════════════════════════

Periodo vero (classico): r = 4

Misura       Decimale   Fase         Periodo    Conteggi  
------------------------------------------------------------
00000000     0          0.0000       -          1050      
11111111     255        0.9961       1          836       
10000000     128        0.5000       2          796  

In [28]:
# Visualizza istogramma
if result_15:
    plot_histogram(result_15['counts'], title='Shor: a=7, N=15')

In [19]:
# ═══════════════════════════════════════════════════════════════
# ESEMPIO 2: Fattorizzare 21 con a=2
# ═══════════════════════════════════════════════════════════════

print("\n" + "▓"*70)
print("ESEMPIO 2: N = 21, a = 2")
print("▓"*70)

result_21 = run_shor_general(N=21, a=2, n_count=10, shots=4096)


▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
ESEMPIO 2: N = 21, a = 2
▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓

════════════════════════════════════════════════════════════
ALGORITMO DI SHOR GENERALIZZATO
════════════════════════════════════════════════════════════
Obiettivo: Fattorizzare N = 21
Base: a = 2
Configurazione circuito Shor:
  N = 21, a = 2
  Qubit conteggio: 10
  Qubit target: 5
  Qubit totali: 15

Esecuzione simulazione...
══════════════════════════════════════════════════════════════════════
ANALISI RISULTATI SHOR per a=2, N=21
══════════════════════════════════════════════════════════════════════

Periodo vero (classico): r = 6

Misura       Decimale   Fase         Periodo    Conteggi  
------------------------------------------------------------
0000000000   0          0.0000       -          721       
1010101010   682        0.6660       3          625       
1101010101   853        0.8330       6          598 

In [20]:
if result_21:
    plot_histogram(result_21['counts'], title='Shor: a=2, N=21')

In [21]:
# ═══════════════════════════════════════════════════════════════
# ESEMPIO 3: Fattorizzare 35 = 5 × 7
# ═══════════════════════════════════════════════════════════════

print("\n" + "▓"*70)
print("ESEMPIO 3: N = 35")
print("▓"*70)

result_35 = run_shor_general(N=35, a=3, n_count=12, shots=4096)


▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
ESEMPIO 3: N = 35
▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓

════════════════════════════════════════════════════════════
ALGORITMO DI SHOR GENERALIZZATO
════════════════════════════════════════════════════════════
Obiettivo: Fattorizzare N = 35
Base: a = 3
Configurazione circuito Shor:
  N = 35, a = 3
  Qubit conteggio: 12
  Qubit target: 6
  Qubit totali: 18

Esecuzione simulazione...
══════════════════════════════════════════════════════════════════════
ANALISI RISULTATI SHOR per a=3, N=35
══════════════════════════════════════════════════════════════════════

Periodo vero (classico): r = 12

Misura       Decimale   Fase         Periodo    Conteggi  
------------------------------------------------------------
000000000000 0          0.0000       -          329       
010101010101 1365       0.3333       3          322       
101010101010 2730       0.6665       3          317       

In [22]:
if result_35:
    plot_histogram(result_35['counts'], title='Shor: a=3, N=35')

In [23]:
# ═══════════════════════════════════════════════════════════════
# ESEMPIO 4: Scelta automatica della base
# ═══════════════════════════════════════════════════════════════

print("\n" + "▓"*70)
print("ESEMPIO 4: N = 33, base automatica")
print("▓"*70)

result_33 = run_shor_general(N=33, n_count=12, shots=4096)  # a scelto automaticamente


▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
ESEMPIO 4: N = 33, base automatica
▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
Base scelta automaticamente: a = 5

════════════════════════════════════════════════════════════
ALGORITMO DI SHOR GENERALIZZATO
════════════════════════════════════════════════════════════
Obiettivo: Fattorizzare N = 33
Base: a = 5
Configurazione circuito Shor:
  N = 33, a = 5
  Qubit conteggio: 12
  Qubit target: 6
  Qubit totali: 18

Esecuzione simulazione...
══════════════════════════════════════════════════════════════════════
ANALISI RISULTATI SHOR per a=5, N=33
══════════════════════════════════════════════════════════════════════

Periodo vero (classico): r = 10

Misura       Decimale   Fase         Periodo    Conteggi  
------------------------------------------------------------
000000000000 0          0.0000       -          420       
111111111111 4095       0.9998       1          169       
100000

## Funzione Helper per Testare Diverse Basi

In [24]:
def try_multiple_bases(N: int, max_attempts: int = 5, shots: int = 2048) -> dict:
    """
    Prova diverse basi fino a trovare i fattori.
    
    Args:
        N: Numero da fattorizzare
        max_attempts: Massimo numero di tentativi
        shots: Shots per ogni tentativo
    
    Returns:
        Risultato della fattorizzazione
    """
    import random
    
    print(f"Tentativo di fattorizzare N = {N}")
    print(f"Basi coprime disponibili: ", end="")
    coprimes = [x for x in range(2, min(N, 20)) if gcd(x, N) == 1]
    print(coprimes[:10], "..." if len(coprimes) > 10 else "")
    print()
    
    for attempt in range(max_attempts):
        # Scegli base
        a = random.choice(coprimes)
        print(f"\n--- Tentativo {attempt + 1}: a = {a} ---")
        
        result = run_shor_general(N, a=a, shots=shots, verbose=False)
        
        if result and result.get('analysis', {}).get('factors'):
            f1, f2 = result['analysis']['factors']
            if f1:
                print(f"\n✓ SUCCESSO! Fattori trovati: {f1} × {N//f1 if f2 is None else f2} = {N}")
                return result
        
        print(f"  Nessun fattore trovato con a={a}")
    
    print(f"\n✗ Nessun fattore trovato dopo {max_attempts} tentativi")
    return None

# Test
print("="*70)
print("TEST: Fattorizzazione con basi multiple")
print("="*70)
result = try_multiple_bases(55, max_attempts=3, shots=2048)

TEST: Fattorizzazione con basi multiple
Tentativo di fattorizzare N = 55
Basi coprime disponibili: [2, 3, 4, 6, 7, 8, 9, 12, 13, 14] ...


--- Tentativo 1: a = 19 ---
Configurazione circuito Shor:
  N = 55, a = 19
  Qubit conteggio: 12
  Qubit target: 6
  Qubit totali: 18
══════════════════════════════════════════════════════════════════════
ANALISI RISULTATI SHOR per a=19, N=55
══════════════════════════════════════════════════════════════════════

Periodo vero (classico): r = 10

Misura       Decimale   Fase         Periodo    Conteggi  
------------------------------------------------------------
000000000000 0          0.0000       -          214       
100000000000 2048       0.5000       2          95        
111111111111 4095       0.9998       1          69        
110011001100 3276       0.7998       5          29        
101100110011 2867       0.7000       10         28        
010110011001 1433       0.3499       20         22        
001100110011 819        0.2000       5 

## Limitazioni e Note

### Limitazioni del Metodo della Matrice di Permutazione

1. **Memoria**: La matrice ha dimensione $2^n \times 2^n$ dove $n = \lceil \log_2(N) \rceil$
   - N=15: matrice 16×16 ✓
   - N=63: matrice 64×64 ✓
   - N=127: matrice 128×128 ✓
   - N=255: matrice 256×256 ⚠ (pesante)
   - N>1000: impraticabile con questo metodo

2. **Profondità circuito**: I gate unitari generici vengono decomposti in molti gate elementari

### Per N Grandi: Circuiti Aritmetici

Per fattorizzare numeri grandi (come RSA-2048), servono circuiti aritmetici:
- **Draper Adder**: Addizione usando QFT
- **VBE Multiplier**: Moltiplicazione con gate di Toffoli
- **Beauregard Circuit**: Ottimizzato per Shor

Questi metodi usano più qubit ausiliari ma scalano polinomialmente con $\log(N)$.

---

**Versione**: 2.2.0 (Generalizzato)  
**Note**: Funziona per N fino a ~128 con il metodo della matrice di permutazione